In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F


import matplotlib.pyplot as plt
#%matplotlib inline

In [2]:
#create Spark session
spark = SparkSession.builder.appName('ChicagoTaxiML').getOrCreate()

#change configuration settings on Spark 
#conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '5g'), ('spark.app.name', 'Spark Updated Conf'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.driver.memory','8g')])

#print spark configuration settings
spark.sparkContext.getConf().getAll()

Out[ 3 ]: 
[('spark.databricks.preemption.enabled', 'true'),
 ('spark.hadoop.fs.abfs.impl',
 'shaded.databricks.v20180920_b33d810.org.apache.hadoop.fs.azurebfs.AzureBlobFileSystem'),
 ('spark.driver.tempDirectory', '/local_disk0/tmp'),
 ('spark.hadoop.fs.adl.impl.disable.cache', 'true'),
 ('spark.hadoop.fs.s3a.connection.maximum', '200'),
 ('spark.databricks.clusterUsageTags.numPerClusterInitScriptsV2', '0'),
 ('spark.executor.memory', '7284m'),
 ('spark.shuffle.reduceLocality.enabled', 'false'),
 ('spark.sql.streaming.checkpointFileManagerClass',
 'com.databricks.spark.sql.streaming.DatabricksCheckpointFileManager'),
 ('spark.hadoop.spark.sql.sources.outputCommitterClass',
 'com.databricks.backend.daemon.data.client.MapReduceDirectOutputCommitter'),
 ('spark.streaming.driver.writeAheadLog.allowBatching', 'true'),
 ('spark.databricks.clusterSource', 'UI'),
 ('spark.databricks.clusterUsageTags.clusterName', 'azurespark'),
 ('spark.databricks.clusterUsageTags.clusterOwnerUserId', '3654560352704485'),
 ('spark.hadoop.hive.server2.transport.mode', 'http'),
 ('spark.sql.parquet.compression.codec', 'snappy'),
 ('spark.databricks.cloudProvider', 'Azure'),
 ('spark.databricks.clusterUsageTags.clusterLogDeliveryEnabled', 'false'),
 ('spark.databricks.clusterUsageTags.containerType', 'LXC'),
 ('spark.eventLog.enabled', 'false'),
 ('spark.hadoop.fs.wasb.impl',
 'shaded.databricks.org.apache.hadoop.fs.azure.NativeAzureFileSystem'),
 ('spark.executor.tempDirectory', '/local_disk0/tmp'),
 ('spark.hadoop.mapred.output.committer.class',
 'com.databricks.backend.daemon.data.client.DirectOutputCommitter'),
 ('spark.databricks.clusterUsageTags.clusterTargetWorkers', '2'),
 ('spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version', '2'),
 ('spark.databricks.clusterUsageTags.driverNodeType', 'Standard_DS3_v2'),
 ('spark.sql.allowMultipleContexts', 'false'),
 ('spark.databricks.clusterUsageTags.driverInstancePrivateIp', '10.139.0.6'),
 ('spark.hadoop.hive.server2.thrift.http.port', '10000'),
 ('spark.home', '/databricks/spark'),
 ('spark.sql.warehouse.dir', '/user/hive/warehouse'),
 ('spark.hadoop.hive.server2.idle.operation.timeout', '7200000'),
 ('spark.task.reaper.enabled', 'true'),
 ('spark.databricks.passthrough.s3a.tokenProviderClassName',
 'com.databricks.backend.daemon.driver.aws.AwsCredentialContextTokenProvider'),
 ('spark.storage.memoryFraction', '0.5'),
 ('spark.databricks.session.share', 'false'),
 ('spark.driver.port', '42633'),
 ('spark.databricks.clusterUsageTags.clusterResourceClass', 'default'),
 ('spark.driver.maxResultSize', '4g'),
 ('spark.executor.extraClassPath',
 '/databricks/spark/dbconf/log4j/executor:/databricks/spark/dbconf/jets3t/:/databricks/spark/dbconf/hadoop:/databricks/hive/conf:/databricks/jars/api-base--api-base_java-spark_2.4_2.11_deploy.jar:/databricks/jars/api-base--api-base-spark_2.4_2.11_deploy.jar:/databricks/jars/api--rpc--rpc_parser-spark_2.4_2.11_deploy.jar:/databricks/jars/chauffeur-api--api--endpoints--endpoints-spark_2.4_2.11_deploy.jar:/databricks/jars/chauffeur-api--chauffeur-api-spark_2.4_2.11_deploy.jar:/databricks/jars/common--client--client-spark_2.4_2.11_deploy.jar:/databricks/jars/common--common-spark_2.4_2.11_deploy.jar:/databricks/jars/common--credentials--credentials-spark_2.4_2.11_deploy.jar:/databricks/jars/common--hadoop--hadoop-spark_2.4_2.11_deploy.jar:/databricks/jars/common--jetty--client--client-spark_2.4_2.11_deploy.jar:/databricks/jars/common--lazy--lazy-spark_2.4_2.11_deploy.jar:/databricks/jars/common--libcommon_resources.jar:/databricks/jars/common--path--path-spark_2.4_2.11_deploy.jar:/databricks/jars/common--rate-limiter--rate-limiter-spark_2.4_2.11_deploy.jar:/databricks/jars/common--storage--storage-spark_2.4_2.11_deploy.jar:/databricks/jars/daemon--data--client--client-spark_2.4_2.11_deploy.jar:/databricks/jars/daemon--data--client--conf--conf-spark_2.4_2.11_deploy.jar:/databricks/jars/daemon--data--client--utils-spark_2.4_2.11_deploy.jar:/databricks/jars/daemon--dat

In [3]:
datadir = "/FileStore/tables/"
cleaned_df = spark.read.csv(datadir + 'FInal_Cleaned_All.csv',header=True, inferSchema=True)

# cleaned_df = spark.read \
#     .option("quote", "\"")  \
#     .option("escape", "\"") \
#     .option("ignoreLeadingWhiteSpace",True) \
#     .csv("/user/atal/data/FInal_Cleaned_All.csv",inferSchema=True, header=True )

In [4]:
cleaned_df.printSchema()

root
-- Date: string (nullable = true)
-- AvgTemp: double (nullable = true)
-- Divvy_TotalTrips: integer (nullable = true)
-- Taxi_total: integer (nullable = true)
-- L_CTA_Sum_rides: integer (nullable = true)
-- Community_Area_Number: integer (nullable = true)
-- Community_Name: string (nullable = true)

In [5]:
cleaned_df =cleaned_df.fillna(0)

In [6]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator

#convert relevant categorical into one hot encoded
indexer1 = StringIndexer(inputCol="Community_Name", outputCol="communityIdx").setHandleInvalid("skip")

#gather all indexers as inputs to the One Hot Encoder
inputs = [indexer1.getOutputCol()]

#create the one hot encoder
encoder = OneHotEncoderEstimator(inputCols=inputs,  \
                                 outputCols=["communityVec"])

#run it through a pipeline
pipeline = Pipeline(stages=[indexer1, encoder])
encodedData = pipeline.fit(cleaned_df).transform(cleaned_df)


encodedData.show(5)

+---------+-----------+----------------+----------+---------------+---------------------+--------------+------------+---------------+
 Date| AvgTemp|Divvy_TotalTrips|Taxi_total|L_CTA_Sum_rides|Community_Area_Number|Community_Name|communityIdx| communityVec|
+---------+-----------+----------------+----------+---------------+---------------------+--------------+------------+---------------+
5/12/2015| 51.0125| 5| 139| 632829| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|
5/13/2015| 45.6125| 6| 174| 644592| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|
5/14/2015| 63.0| 1| 155| 653295| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|
5/15/2015| 64.2| 12| 220| 661165| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|
5/16/2015|71.49833333| 13| 244| 397503| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|
+---------+-----------+----------------+----------+---------------+---------------------+--------------+------------+---------------+
only showing top 5 rows

In [7]:
from pyspark.ml.feature import VectorAssembler

#gather feature vector and identify features
assembler = VectorAssembler(inputCols = ['AvgTemp', 'Divvy_TotalTrips', \
                                          'L_CTA_Sum_rides', 'communityVec'], \
                            outputCol = 'features')

encodedData = assembler.transform(encodedData)

In [8]:
encodedData.show()

+---------+-----------+----------------+----------+---------------+---------------------+--------------+------------+---------------+--------------------+
 Date| AvgTemp|Divvy_TotalTrips|Taxi_total|L_CTA_Sum_rides|Community_Area_Number|Community_Name|communityIdx| communityVec| features|
+---------+-----------+----------------+----------+---------------+---------------------+--------------+------------+---------------+--------------------+
5/12/2015| 51.0125| 5| 139| 632829| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[5...|
5/13/2015| 45.6125| 6| 174| 644592| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[4...|
5/14/2015| 63.0| 1| 155| 653295| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[6...|
5/15/2015| 64.2| 12| 220| 661165| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[6...|
5/16/2015|71.49833333| 13| 244| 397503| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[7...|
5/17/2015|77.37380952| 28| 216| 302360| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[7...|
5/18/2015|69.11520408| 26| 169| 626097| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[6...|
5/19/2015|47.53860544| 22| 168| 624394| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[4...|
5/20/2015| 44.575| 11| 181| 624789| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[4...|
5/21/2015|58.57638889| 23| 189| 644807| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[5...|
5/22/2015|56.41802721| 24| 252| 624758| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[5...|
5/23/2015| 71.271875| 34| 248| 404353| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[7...|
5/24/2015|67.43163265| 18| 264| 296032| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[6...|
5/25/2015|74.15833333| 21| 152| 288930| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[7...|
5/26/2015| 69.3912037| 39| 172| 610732| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[6...|
5/27/2015|70.01119048| 41| 148| 643974| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[7...|
5/28/2015| 74.55| 26| 148| 646731| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[7...|
5/29/2015|76.44791667| 29| 196| 657746| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[7...|
5/30/2015| 55.96| 8| 279| 345204| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[5...|
5/31/2015|50.90555556| 14| 220| 284022| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[5...|
+---------+-----------+----------------+----------+---------------+---------------------+--------------+------------+---------------+--------------------+
only showing top 20 rows

In [9]:
encodedData = encodedData.withColumn('trip_date',F.from_unixtime(F.unix_timestamp(encodedData.Date,format='MM/dd/yyyy'),format ='yyyy-MM-dd').cast('date').alias('trip_date'))

In [10]:
#split data into train and test
train_df = encodedData.filter(encodedData.trip_date <='2016-12-31')
train_df = train_df.fillna(0)
train_df.show()

+---------+-----------+----------------+----------+---------------+---------------------+--------------+------------+---------------+--------------------+----------+
 Date| AvgTemp|Divvy_TotalTrips|Taxi_total|L_CTA_Sum_rides|Community_Area_Number|Community_Name|communityIdx| communityVec| features| trip_date|
+---------+-----------+----------------+----------+---------------+---------------------+--------------+------------+---------------+--------------------+----------+
5/12/2015| 51.0125| 5| 139| 632829| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[5...|2015-05-12|
5/13/2015| 45.6125| 6| 174| 644592| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[4...|2015-05-13|
5/14/2015| 63.0| 1| 155| 653295| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[6...|2015-05-14|
5/15/2015| 64.2| 12| 220| 661165| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[6...|2015-05-15|
5/16/2015|71.49833333| 13| 244| 397503| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[7...|2015-05-16|
5/17/2015|77.37380952| 28| 216| 302360| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[7...|2015-05-17|
5/18/2015|69.11520408| 26| 169| 626097| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[6...|2015-05-18|
5/19/2015|47.53860544| 22| 168| 624394| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[4...|2015-05-19|
5/20/2015| 44.575| 11| 181| 624789| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[4...|2015-05-20|
5/21/2015|58.57638889| 23| 189| 644807| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[5...|2015-05-21|
5/22/2015|56.41802721| 24| 252| 624758| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[5...|2015-05-22|
5/23/2015| 71.271875| 34| 248| 404353| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[7...|2015-05-23|
5/24/2015|67.43163265| 18| 264| 296032| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[6...|2015-05-24|
5/25/2015|74.15833333| 21| 152| 288930| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[7...|2015-05-25|
5/26/2015| 69.3912037| 39| 172| 610732| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[6...|2015-05-26|
5/27/2015|70.01119048| 41| 148| 643974| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[7...|2015-05-27|
5/28/2015| 74.55| 26| 148| 646731| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[7...|2015-05-28|
5/29/2015|76.44791667| 29| 196| 657746| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[7...|2015-05-29|
5/30/2015| 55.96| 8| 279| 345204| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[5...|2015-05-30|
5/31/2015|50.90555556| 14| 220| 284022| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[5...|2015-05-31|
+---------+-----------+----------------+----------+---------------+---------------------+--------------+------------+---------------+--------------------+----------+
only showing top 20 rows

In [11]:
test_df = encodedData.filter(encodedData.trip_date >'2016-12-31')
test_df = test_df.na.fill(0)
test_df.show()

+---------+-----------+----------------+----------+---------------+---------------------+--------------+------------+---------------+--------------------+----------+
 Date| AvgTemp|Divvy_TotalTrips|Taxi_total|L_CTA_Sum_rides|Community_Area_Number|Community_Name|communityIdx| communityVec| features| trip_date|
+---------+-----------+----------------+----------+---------------+---------------------+--------------+------------+---------------+--------------------+----------+
 1/1/2017|32.01666667| 18| 104| 237792| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[3...|2017-01-01|
 1/2/2017|38.30277778| 15| 44| 255944| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[3...|2017-01-02|
 1/3/2017|36.10121693| 28| 56| 532948| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[3...|2017-01-03|
 1/4/2017|12.12878788| 18| 59| 524200| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[1...|2017-01-04|
 1/5/2017|8.522916667| 18| 68| 515984| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[8...|2017-01-05|
 1/6/2017|4.731666667| 18| 72| 491688| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[4...|2017-01-06|
 1/7/2017|10.95185185| 12| 64| 256830| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[1...|2017-01-07|
 1/8/2017|12.60208333| 16| 58| 198431| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[1...|2017-01-08|
 1/9/2017|25.02564103| 23| 53| 567454| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[2...|2017-01-09|
1/10/2017|37.16904762| 26| 52| 582540| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[3...|2017-01-10|
1/11/2017|34.44270833| 15| 60| 585290| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[3...|2017-01-11|
1/12/2017| 22.975| 22| 48| 585012| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[2...|2017-01-12|
1/13/2017|20.26318182| 24| 62| 571941| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[2...|2017-01-13|
1/14/2017| 30.89| 21| 73| 292018| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[3...|2017-01-14|
1/15/2017| 30.3962963| 19| 84| 228344| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[3...|2017-01-15|
1/16/2017|33.08055556| 10| 31| 356341| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[3...|2017-01-16|
1/17/2017|36.37361111| 23| 43| 604477| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[3...|2017-01-17|
1/18/2017|34.54166667| 32| 58| 607921| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[3...|2017-01-18|
1/19/2017|37.78030303| 28| 59| 608646| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[3...|2017-01-19|
1/20/2017|40.24060606| 32| 59| 599268| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[4...|2017-01-20|
+---------+-----------+----------------+----------+---------------+---------------------+--------------+------------+---------------+--------------------+----------+
only showing top 20 rows

In [12]:
%%time
from pyspark.ml.regression import LinearRegression

#Elastic Net
lr = LinearRegression(featuresCol = 'features', labelCol='Taxi_total', regParam=0.3, elasticNetParam=0.8, maxIter=10)
lrm = lr.fit(train_df)

#coefficients
print("Coefficients: " + str(lrm.coefficients))
print("Intercept: " + str(lrm.intercept))

#model summary
print("RMSE: %f" % lrm.summary.rootMeanSquaredError)
print("r2: %f" % lrm.summary.r2)

#p-values are not provided in this model for the solver being used
#print("pValues: " + str(lrm.summary.pValues))

Coefficients: [-2.33241797272,0.44316480734,0.000284414732116,12530.7591222,7970.57509811,-150.24319308,2719.22323375,1127.2108806,1732.04706015,416.113293359,-609.77483301,-683.92988763,1165.87812354,-464.075783438,-447.647718948,-1003.24808953,-985.80823752,-938.505689206,-1056.79478113,-839.349473622,-1035.45386159,-945.377029126,-1019.51392093,-1163.33374654,-1178.72114654,-1154.27900289,-1091.28547313,-1192.52886653,-1102.42430504,-1141.2334924,-1074.71913457,-1119.93302525,-1139.33846622,-1204.91521381,-1154.59408197,-1108.21838142,-1154.56766612,-1124.67186557,-1195.31485231,-1116.60701767,-1156.07134308,-1199.6804583,-1122.51293692,-1128.92820844,-1156.71675339,-1185.11026359,-1209.25802241,-1162.12505068,-1219.14495085,-1214.41800051,-1217.86609411,-1220.10871833,-1225.61636715,-1217.97705202,-1220.17733081,-1224.23995911,-1224.97542135,-1208.36086889,-1221.62445019,-1223.73688641,-1226.32066448,-1226.90484257,-1224.55116431,-1222.52465395,-1222.82257053,-1221.76513964,-1230.90527844,-1226.07575015,-1221.53235614,-1224.85126176,-1225.25685791,-1228.91030853,-1220.40633732,-1224.64051814,-1160.3308035,-1231.67044294,-1214.26761981,-1224.1895531,-1228.8942782]
Intercept: 1101.8816807840485
RMSE: 1119.545170
r2: 0.891487
CPU times: user 30.2 ms, sys: 12.5 ms, total: 42.7 ms
Wall time: 7.23 s

In [13]:
#make predictions
predictions = lrm.transform(test_df)

In [14]:
from pyspark.ml.evaluation import RegressionEvaluator

eval = RegressionEvaluator(labelCol="Taxi_total", predictionCol="prediction", metricName="rmse")

# Root Mean Square Error
rmse = eval.evaluate(predictions)
print("RMSE: %.3f" % rmse)

# Mean Square Error
mse = eval.evaluate(predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval.evaluate(predictions, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval.evaluate(predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)

RMSE: 2745.040
MSE: 7535245.588
MAE: 1033.385
r2: -4.879

In [15]:
predictions.show()

+---------+-----------+----------------+----------+---------------+---------------------+--------------+------------+---------------+--------------------+----------+------------------+
 Date| AvgTemp|Divvy_TotalTrips|Taxi_total|L_CTA_Sum_rides|Community_Area_Number|Community_Name|communityIdx| communityVec| features| trip_date| prediction|
+---------+-----------+----------------+----------+---------------+---------------------+--------------+------------+---------------+--------------------+----------+------------------+
 1/1/2017|32.01666667| 18| 104| 237792| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[3...|2017-01-01|117.00570900764569|
 1/2/2017|38.30277778| 15| 44| 255944| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[3...|2017-01-02|106.17707227152687|
 1/3/2017|36.10121693| 28| 56| 532948| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[3...|2017-01-03|195.85719331665757|
 1/4/2017|12.12878788| 18| 59| 524200| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[1...|2017-01-04|244.85120953265618|
 1/5/2017|8.522916667| 18| 68| 515984| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[8...|2017-01-05|250.92485691810043|
 1/6/2017|4.731666667| 18| 72| 491688| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[4...|2017-01-06| 252.8574962256747|
 1/7/2017|10.95185185| 12| 64| 256830| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[1...|2017-01-07|168.89336051180157|
 1/8/2017|12.60208333| 16| 58| 198431| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[1...|2017-01-08|150.20745423720518|
 1/9/2017|25.02564103| 23| 53| 567454| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[2...|2017-01-09|229.28825631373525|
1/10/2017|37.16904762| 26| 52| 582540| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[3...|2017-01-10|206.58493160391038|
1/11/2017|34.44270833| 15| 60| 585290| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[3...|2017-01-11|208.85122199621514|
1/12/2017| 22.975| 22| 48| 585012| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[2...|2017-01-12| 238.6217973668597|
1/13/2017|20.26318182| 24| 62| 571941| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[2...|2017-01-13|242.11563547982644|
1/14/2017| 30.89| 21| 73| 292018| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[3...|2017-01-14|136.38573428377424|
1/15/2017| 30.3962963| 19| 84| 228344| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[3...|2017-01-15|118.54110439940064|
1/16/2017|33.08055556| 10| 31| 356341| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[3...|2017-01-16|144.69603905856422|
1/17/2017|36.37361111| 23| 43| 604477| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[3...|2017-01-17|213.34993357240558|
1/18/2017|34.54166667| 32| 58| 607921| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[3...|2017-01-18|222.59080131275368|
1/19/2017|37.78030303| 28| 59| 608646| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[3...|2017-01-19|213.47048911101206|
1/20/2017|40.24060606| 32| 59| 599268| 1| ROGERS PARK| 13.0|(76,[13],[1.0])|(79,[0,1,2,16],[4...|2017-01-20| 206.8374519770789|
+---------+-----------+----------------+----------+---------------+---------------------+--------------+------------+---------------+--------------------+----------+------------------+
only showing top 20 rows

In [16]:
#view predictions against test
predictions.select("Taxi_Total","prediction").show(365)

+----------+-------------------+
Taxi_Total| prediction|
+----------+-------------------+
 104| 117.00570900764569|
 44| 106.17707227152687|
 56| 195.85719331665757|
 59| 244.85120953265618|
 68| 250.92485691810043|
 72| 252.8574962256747|
 64| 168.89336051180157|
 58| 150.20745423720518|
 53| 229.28825631373525|
 52| 206.58493160391038|
 60| 208.85122199621514|
 48| 238.6217973668597|
 62| 242.11563547982644|
 73| 136.38573428377424|
 84| 118.54110439940064|
 31| 144.69603905856422|
 43| 213.34993357240558|
 58| 222.59080131275368|
 59| 213.47048911101206|
 59| 206.8374519770789|
 66| 149.04418529459508|
 69| 92.95400699820618|
 39| 216.42799299042304|
 40| 220.82907915923965|
 45| 214.85992681083678|
 59| 232.09364550097723|
 75| 238.65028232336306|
 77| 148.79322842840645|
 92| 125.5199588645994|
 37| 235.44334166196552|
 53| 216.21145708627137|
 61| 227.48986372857337|
 66| 260.3513525930206|
 65| 238.9295682123278|
 80| 143.82061641425184|
 60| 120.36716756144631|
 54| 205.6824742215996|
 43| 201.98208582853613|
 50| 235.834925620871|
 40| 252.76878967960613|
 54| 225.09765242120886|
 66| 121.02733520780112|
 58| 100.5799466827292|
 51| 213.6442438148987|
 48| 206.99951092606761|
 41| 227.45851149153646|
 50| 218.67174683333656|
 52| 187.80871304037703|
 74| 104.26121868881262|
 92| 73.03495105263914|
 56| 126.44895304053796|
 63| 172.61988790755254|
 59| 180.91644550707474|
 55| 203.03800208615166|
 68| 206.94824483448656|
 110| 148.77176721237083|
 71| 93.48624827077026|
 72| 207.18105665143253|
 52| 178.1508192582353|
 65| 208.45642853985817|
 68| 231.27099926008862|
 66| 236.56094547449402|
 67| 134.9324326052223|
 113| 78.6047241364115|
 55| 164.9747195471416|
 36| 202.80546266359886|
 58| 202.0106396038425|
 61| 220.44248525568457|
 82| 242.2776123444728|
 81| 175.94757408492433|
 97| 127.3772311362111|
 67| 224.04717814642243|
 66| 240.0484635952123|
 55| 244.1324850534968|
 63| 235.1303464706159|
 81| 207.59468392752035|
 83| 128.80642874836838|
 89| 93.88518822577839|
 51| 188.1547018076459|
 56| 209.6479699651984|
 58| 224.84281278992125|
 65| 205.91284958366884|
 62| 172.85314542202548|
 82| 104.57097977749845|
 89| 62.5759702964333|
 53| 194.48438179638185|
 64| 214.7525928162678|
 53| 210.9427303959286|
 71| 194.2783895123314|
 72| 213.7278342588188|
 89| 121.46639535168424|
 103| 83.05108447327734|
 61| 186.9223085616943|
 70| 196.72937010777855|
 26| 190.90229868981794|
 43| 214.09994941705634|
 73| 208.55569737609517|
 81| 98.35872629912251|
 92| 58.186391467973635|
 64| 140.02060130493123|
 56| 190.89335152669503|
 50| 218.14004082922793|
 61| 190.34435483719767|
 58| 161.43059427789694|
 77| 79.43416079423059|
 85| 50.589479730281255|
 60| 179.02437453817276|
 68| 188.56620280819982|
 55| 184.38232919708264|
 57| 184.63110547203962|
 68| 207.7823060764597|
 70| 119.316034348421|
 93| 82.37883362390687|
 4| 176.1756248214641|
 11| 170.0363962057845|
 3| 167.8514887906873|
 8| 203.50895741601323|
 26| 203.3975318891928|
 6| 108.87233097757633|
 11| 79.75921815684592|
 11| 188.7202897716736|
 9| 210.61126464460972|
 9| 210.9954357420827|
 6| 218.8466448718824|
 15| 201.48912841361778|
 15| 120.11833812428733|
 37| 93.70985361391138|
 17| 196.5182397018723|
 10| 191.63717246174986|
 8| 175.77785871469678|
 6| 197.3627379458269|
 4| 177.72882698431226|
 9| 76.0320678779824|
 8| 71.41905779839362|
 5| 153.26156412627245|
 6| 149.5067277012438|
 4| 148.26985216980688|
 2| 171.11614269993743|
 3| 198.39648554819837|
 4| 83.02625238265102|
 9| 91.91857619050597|
 4| 174.7678967124359|
 4| 174.42109836632596|
 1| 184.47766602280285|
 4| 190.68699168616558|
 9| 151.00888727546862|
 10| 92.85444227652579|
 8| 54.73715433859002|
 3| 49.33248646519837|
 4| 163.14835503372944|
 5| 176.86589925735996|
 4| 172.58884485871909|
 3| 167.3195750523089|
 12| 83.98267587409305|
 7| 54.105092978995344|
 6| 178.86765156507272|
 4| 181.83673073102568|
 6| 185.4335865164062|
 7| 171.70475157184217|
 6| 186.54547416940716|
 7| 80.0

In [17]:
display(predictions.select("Taxi_Total","prediction"))

Taxi_Total,prediction
104,117.00570900764569
44,106.17707227152687
56,195.85719331665757
59,244.85120953265618
68,250.92485691810043
72,252.8574962256747
64,168.89336051180157
58,150.20745423720518
53,229.28825631373525
52,206.58493160391038
